In [ ]:
# Import necessary libraries
import requests
import time
import yaml

# Function to load configuration from a YAML file
def load_config(filepath="config.yaml"):
    """
    Loads API credentials and other settings from a configuration file.
    """
    try:
        with open(filepath, "r") as file:
            return yaml.safe_load(file)
    except Exception as e:
        print(f"Error loading config: {e}")
        raise

# Load the configuration
config = load_config()

# Display a confirmation message
print("Configuration loaded successfully!")

In [ ]:
# Function to search for a beer by name
def search_beer(api_config, beer_name):
    """
    Search for beers by name using the Untappd API.
    Returns a list of matching beers.
    """
    url = f"{api_config['base_url']}/search/beer"
    params = {
        "client_id": api_config["client_id"],
        "client_secret": api_config["client_secret"],
        "q": beer_name,
    }

    while True:
        response = requests.get(url, params=params)
        print(f"Searching for beer: {beer_name}... Response: {response.status_code}")
        
        if response.status_code == 429:  # Handle rate limiting
            print("Rate limit reached. Waiting for 60 seconds...")
            time.sleep(60)
            continue

        response.raise_for_status()
        return response.json()["response"]["beers"]["items"]

# Prompt the user to enter a beer name
beer_name = input("Enter the name of the beer you want to search for: ")

# Search for beers
matching_beers = search_beer(config["api"], beer_name)

# Display the results
if matching_beers:
    print("\nMatching Beers:")
    for idx, beer in enumerate(matching_beers, 1):
        print(f"{idx}. {beer['beer']['beer_name']} - {beer['brewery']['brewery_name']}")
else:
    print("No matching beers found.")

In [ ]:
# Function to fetch detailed beer information
def fetch_beer_details(api_config, beer_id):
    """
    Fetch detailed information about a beer by its ID.
    """
    url = f"{api_config['base_url']}/beer/info/{beer_id}"
    params = {
        "client_id": api_config["client_id"],
        "client_secret": api_config["client_secret"],
    }

    while True:
        response = requests.get(url, params=params)
        print(f"Fetching details for beer_id {beer_id}... Response: {response.status_code}")
        
        if response.status_code == 429:  # Handle rate limiting
            print("Rate limit reached. Waiting for 60 seconds...")
            time.sleep(60)
            continue

        response.raise_for_status()
        return response.json()["response"]["beer"]

# Allow the user to select a beer
if matching_beers:
    selected_index = int(input("\nEnter the number of the beer to fetch details: ")) - 1
    if 0 <= selected_index < len(matching_beers):
        selected_beer = matching_beers[selected_index]
        beer_id = selected_beer["beer"]["bid"]
        beer_details = fetch_beer_details(config["api"], beer_id)

        # Display beer details
        print("\nBeer Details:")
        print(f"Name: {beer_details['beer_name']}")
        print(f"Style: {beer_details['beer_style']}")
        print(f"ABV: {beer_details['beer_abv']}%")
        print(f"IBU: {beer_details['beer_ibu']}")
        print(f"Description: {beer_details['beer_description']}")
    else:
        print("Invalid selection.")

In [ ]:
# Import necessary libraries for database operations
import psycopg2
import pandas as pd

# Function to connect to the database and retrieve beer ratings
def fetch_rated_beers_from_db(db_config):
    """
    Connects to the PostgreSQL database and retrieves historical beer ratings and attributes.
    Returns the data as a Pandas DataFrame.
    """
    query = """
    SELECT 
        beer_id, beer_name, style, abv, ibu, rating_score, user_rating
    FROM 
        rated_beers
    """
    try:
        # Connect to the database
        connection = psycopg2.connect(**db_config)
        print("Connected to the database successfully.")
        
        # Execute the query and load results into a DataFrame
        df = pd.read_sql_query(query, connection)
        return df

    except Exception as e:
        print(f"Error fetching data from the database: {e}")
        raise

    finally:
        if connection:
            connection.close()

# Fetch beer ratings data
rated_beers_df = fetch_rated_beers_from_db(config["database"])

# Display the first few rows of the DataFrame
print("Fetched beer ratings data:")
print(rated_beers_df.head())

In [ ]:
# Analyze average ratings by style
style_preferences = rated_beers_df.groupby("style")["user_rating"].mean().sort_values(ascending=False)

print("\nYour Average Ratings by Beer Style:")
print(style_preferences)

# Analyze average ratings by ABV
abv_preferences = rated_beers_df.copy()
abv_preferences["abv_range"] = pd.cut(abv_preferences["abv"], bins=[0, 4, 6, 8, 10, 15], 
                                      labels=["0-4%", "4-6%", "6-8%", "8-10%", "10-15%"])
abv_preferences = abv_preferences.groupby("abv_range")["user_rating"].mean()

print("\nYour Average Ratings by ABV Range:")
print(abv_preferences)

# Analyze average ratings by IBU (bitterness)
ibu_preferences = rated_beers_df.copy()
ibu_preferences["ibu_range"] = pd.cut(ibu_preferences["ibu"], bins=[0, 20, 40, 60, 80, 100], 
                                      labels=["0-20", "20-40", "40-60", "60-80", "80-100"])
ibu_preferences = ibu_preferences.groupby("ibu_range")["user_rating"].mean()

print("\nYour Average Ratings by IBU Range:")
print(ibu_preferences)

In [ ]:
# Import necessary libraries for visualization
import matplotlib.pyplot as plt

# Function to create bar charts
def plot_preferences(data, title, xlabel, ylabel):
    """
    Plots a bar chart for the given data.
    """
    plt.figure(figsize=(10, 6))
    data.plot(kind="bar", color="skyblue", edgecolor="black")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=45)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

# Plot preferences by style
plot_preferences(style_preferences, 
                 title="Average Ratings by Beer Style", 
                 xlabel="Beer Style", 
                 ylabel="Average Rating")

# Plot preferences by ABV range
plot_preferences(abv_preferences, 
                 title="Average Ratings by ABV Range", 
                 xlabel="ABV Range", 
                 ylabel="Average Rating")

# Plot preferences by IBU range
plot_preferences(ibu_preferences, 
                 title="Average Ratings by IBU Range", 
                 xlabel="IBU Range", 
                 ylabel="Average Rating")